In [1]:
import dask
import dask.threaded
import dask.multiprocessing
from dask.distributed import Client

c = Client()


/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/dashboard/core.py:74: UserWarning: 
Failed to start diagnostics server on port 8787. [Errno 13] Permission denied
  warnings.warn("\n" + msg)


In [2]:
c

Client Scheduler: tcp://127.0.0.1:35052 Dashboard: http://127.0.0.1:38049/status,Cluster Workers: 8 Cores: 48 Memory: 134.91 GB


In [3]:
import xarray as xr 
import dask 
import numpy as np 
import os 
import time 
import glob
from datetime import date
import numpy.ma as ma

today=date.today()

#https://pmall.univ-grenoble-alpes.fr/docs/auto_examples_code/windrose/windrose-exemple.html

In [4]:
def dx_var(data,e1):
    dx_var = (data.shift(x=-1) - data)/e1
    return dx_var
def dy_var(data,e2):
    dy_var = (data.shift(y=-1) - data)/e2
    return dy_var
def alpha_beta(tem,salt):
    zt=tem-10  #potential temperature anomaly
    zs=salt-35 #abs salinity anomaly
    zh=0.4804548 # depth at the surface
    rn_a0=1.6550e-1 # thermal expansion coeff
    rn_b0=7.6554e-1 # saline  expansion coeff
    rn_lambda1=5.9520e-2 #cabbeling coeff. in T^2
    rn_mu1=1.4970e-4 #thermobaric coeff. in T
    rn_nu=2.4341e-3 #cabbeling coeff. in theta*salt  
    rau0=1026  #volumic mass of reference     [kg/m3]
    r1_rau0= 1/rau0
    rn_lambda2=5.4914e-4 #cabbeling coeff. in S^2
    rn_mu2=1.1090e-5 #thermobaric coeff. in S
    zn  = rn_a0 * ( 1. + rn_lambda1*zt + rn_mu1*zh ) + rn_nu*zs
    alpha=zn * r1_rau0 
    zn  = rn_b0 * ( 1. - rn_lambda2*zs - rn_mu2*zh ) - rn_nu*zt
    beta=zn * r1_rau0
    return alpha,beta

In [5]:
data_dir = '/store/molines/NATL60/NATL60-CJM165-S/1d/'
gridfile='/store/molines/NATL60/NATL60-I/NATL60_v4.1_cdf_mesh_hgr.nc'
dsgrid=xr.open_dataset(gridfile,chunks={'x':1000,'y':1000})

In [6]:
e1=dsgrid['e1t'][0]
e2=dsgrid['e2t'][0]


In [7]:
imin = {'AC' : 3088, 'NE' :3838, 'GS' : 994}
imax = {'AC' : 3386, 'NE' :4119, 'GS' : 1295}
jmin = {'AC' : 301, 'NE' :1783, 'GS' : 751}
jmax = {'AC' : 657, 'NE' :2239, 'GS' : 1155}


In [8]:
#for month in ['march', 'september']:
for month in ['september']:

    if month == 'march':
        Date=['20130301','20130302','20130303','20130304','20130305','20130306','20130307','20130308','20130309','20130310','20130311','20130312','20130313','20130314','20130315','20130316','20130317','20130318','20130319','20130320','20130321','20130322','20130323','20130324','20130325','20130326','20130327','20130328','20130329','20130330']
    if month == 'september':
        Date=['20120901','20120902','20120903','20120904','20120905','20120906','20120907','20120908','20120909','20120910','20120911','20120912','20120913','20120914','20120915','20120916','20120917','20120918','20120919','20120920','20120921','20120922','20120923','20120924','20120925','20120926','20120927','20120928','20120929','20120930']

    for date in Date:
        print(date)
        
        year=date[0:4]
        month=date[4:6]
        day=date[6:]

        filename = sorted(glob.glob(data_dir+str(year)+'/NATL60-CJM165_y'+str(year)+'m'+str(month)+'d'+str(day)+'.1d_gridTsurf.nc'))
        file=filename[0]
        print(file)

        ds=xr.open_dataset(file,chunks={'x':1000,'y':1000,'time_counter':1})
        dataT=ds['sosstsst']
        dataS=ds['sosaline']
        dataT_dx=dx_var(dataT,e1)
        dataS_dx=dx_var(dataS,e1)
        dataT_dy=dy_var(dataT,e1)
        dataS_dy=dy_var(dataS,e1)
        alpha,beta=alpha_beta(dataT,dataS)
        R=alpha*(dataT_dx + 1j * dataT_dy)/(beta*(dataS_dx + 1j * dataS_dy))
        magR=np.abs(R)
        magR_ma=ma.masked_invalid(magR)
        Tu=np.arctan(magR)
        phi=np.angle(R)
        buoy=np.abs(alpha*(dataT_dx+1j*dataT_dy)-beta*(dataS_dx+1j*dataS_dy))
        spice=np.abs(alpha*(dataT_dx+1j*dataT_dy)+beta*(dataS_dx+1j*dataS_dy))

        for box in ['NE','GS']:
#        for box in ['AC']:

            filename='/scratch/cnt0024/hmg2840/albert7a/NATL60/NATL60-CJM165-S/1d/turner-angle/NATL60-CJM165_grad-buoy_'+str(date)+'.1d_'+str(box)+'.nc'
            if not os.path.exists(filename):
                
                print('compute buoy for box '+str(box))
                magR_box=magR[:,jmin[box]:jmax[box],imin[box]:imax[box]].stack(z=('x', 'y','time_counter'))
                buoy_box=buoy[:,jmin[box]:jmax[box],imin[box]:imax[box]].stack(z=('x', 'y','time_counter'))
        
                magR_box_nonan=magR_box[~np.isnan(magR_box)]
                buoy_box_nonan=buoy_box[~np.isnan(magR_box)]
                magR_box_nonan_noinf=magR_box_nonan[~np.isinf(magR_box_nonan)]
                buoy_box_nonan_noinf=buoy_box_nonan[~np.isinf(magR_box_nonan)]
                magR_box_nonan_noinf=magR_box_nonan_noinf.reset_index('z')
                buoy_box_nonan_noinf=buoy_box_nonan_noinf.reset_index('z')

                print('write '+filename)
                dataset_mag=buoy_box_nonan_noinf.to_dataset(name='buoy')
                dataset_mag.attrs['global_attribute']= 'Stack of grad buoyancy of data for day '+str(date)
                dataset_mag.to_netcdf(path=filename,mode='w')

            filename='/scratch/cnt0024/hmg2840/albert7a/NATL60/NATL60-CJM165-S/1d/turner-angle/NATL60-CJM165_grad-spice_'+str(date)+'.1d_'+str(box)+'.nc'
            if not os.path.exists(filename):
                
                print('compute buoy for box '+str(box))
                magR_box=magR[:,jmin[box]:jmax[box],imin[box]:imax[box]].stack(z=('x', 'y','time_counter'))
                spice_box=spice[:,jmin[box]:jmax[box],imin[box]:imax[box]].stack(z=('x', 'y','time_counter'))
        
                magR_box_nonan=magR_box[~np.isnan(magR_box)]
                spice_box_nonan=spice_box[~np.isnan(magR_box)]
                magR_box_nonan_noinf=magR_box_nonan[~np.isinf(magR_box_nonan)]
                spice_box_nonan_noinf=spice_box_nonan[~np.isinf(magR_box_nonan)]
                magR_box_nonan_noinf=magR_box_nonan_noinf.reset_index('z')
                spice_box_nonan_noinf=spice_box_nonan_noinf.reset_index('z')

                print('write '+filename)
                dataset_mag=spice_box_nonan_noinf.to_dataset(name='spice')
                dataset_mag.attrs['global_attribute']= 'Stack of grad spicyness of data for day '+str(date)
                dataset_mag.to_netcdf(path=filename,mode='w')


20120901
/store/molines/NATL60/NATL60-CJM165-S/1d/2012/NATL60-CJM165_y2012m09d01.1d_gridTsurf.nc
20120902
/store/molines/NATL60/NATL60-CJM165-S/1d/2012/NATL60-CJM165_y2012m09d02.1d_gridTsurf.nc
20120903
/store/molines/NATL60/NATL60-CJM165-S/1d/2012/NATL60-CJM165_y2012m09d03.1d_gridTsurf.nc
20120904
/store/molines/NATL60/NATL60-CJM165-S/1d/2012/NATL60-CJM165_y2012m09d04.1d_gridTsurf.nc
20120905
/store/molines/NATL60/NATL60-CJM165-S/1d/2012/NATL60-CJM165_y2012m09d05.1d_gridTsurf.nc
20120906
/store/molines/NATL60/NATL60-CJM165-S/1d/2012/NATL60-CJM165_y2012m09d06.1d_gridTsurf.nc
20120907
/store/molines/NATL60/NATL60-CJM165-S/1d/2012/NATL60-CJM165_y2012m09d07.1d_gridTsurf.nc
20120908
/store/molines/NATL60/NATL60-CJM165-S/1d/2012/NATL60-CJM165_y2012m09d08.1d_gridTsurf.nc
20120909
/store/molines/NATL60/NATL60-CJM165-S/1d/2012/NATL60-CJM165_y2012m09d09.1d_gridTsurf.nc
20120910
/store/molines/NATL60/NATL60-CJM165-S/1d/2012/NATL60-CJM165_y2012m09d10.1d_gridTsurf.nc
20120911
/store/molines/NATL60

distributed.nanny - WARNING - Worker process 51944 was killed by signal 9
distributed.nanny - WARNING - Worker process 51946 was killed by signal 9
distributed.nanny - WARNING - Worker process 51947 was killed by signal 9
distributed.nanny - WARNING - Worker process 51950 was killed by signal 9
distributed.nanny - WARNING - Worker process 51948 was killed by signal 9
distributed.nanny - WARNING - Worker process 51952 was killed by signal 9
distributed.nanny - WARNING - Worker process 51954 was killed by signal 9
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker process 51958 was killed by signal 9
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


write /scratch/cnt0024/hmg2840/albert7a/NATL60/NATL60-CJM165-S/1d/turner-angle/NATL60-CJM165_grad-spice_20120930.1d_NE.nc
compute buoy for box GS
write /scratch/cnt0024/hmg2840/albert7a/NATL60/NATL60-CJM165-S/1d/turner-angle/NATL60-CJM165_grad-buoy_20120930.1d_GS.nc
compute buoy for box GS
write /scratch/cnt0024/hmg2840/albert7a/NATL60/NATL60-CJM165-S/1d/turner-angle/NATL60-CJM165_grad-spice_20120930.1d_GS.nc


distributed.nanny - WARNING - Worker process 2854 was killed by signal 9
distributed.nanny - WARNING - Worker process 2864 was killed by signal 9
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
